# Project 1 - Locations of charging stations for battery electric vehicles.

## Introduction to the project

### Objective

### Data set

#### Index sets

$dis\_idx \in DIS\_IDX$: Index and set of districts.

$fs\_idx \in FS\_IDX = [0, N\_FIXED]$: Index and set of fixed rental stations.

$ds\_idx \in DS\_IDX = [0,N\_DYNAMIC*3]$: Index and set of dynamic rental stations.

$s\_idx \in S\_IDX = fs\_idx \cup ds\_idx$: Index and set of dynamic rental stations.


$\quad \forall ds\_idx \in DS\_IDX$: Type of $ds\_idx$ = $ds\_idx$ // $N\_DYNAMIC$ (0:'small', 1:'medium', 2:'large')

$\quad \forall ds\_idx \in DS\_IDX$: Position of $ds\_idx$ = $ds\_idx$ % $N\_DYNAMIC$.



#### Parameters

$demand_{dis\_idx} \in \mathbb{R}^+$: Demand for stations in district $dis\_idx$.

$\text{costS} = 10$: Cost of building a small station.

$\text{costM} = 20$: Cost of building a medium station.

$\text{costL} = 50$: Cost of building a large station.

$capS= 10$: Number of bikes for a small station.
 
$capM = 30$: Number of bikes for a medium station.

$capL = 100$: Number of bikes for a large station.

#### Decision variables

$ysmall_{ds\_idx} \in \{0, 1 \}$: 1 if small station built at position of $ds\_idx$; otherwise 0.

$ymed_{ds\_idx} \in \{0, 1 \}$: 1 if medium station built at position of $ds\_idx$; otherwise 0.

$ylarge_{ds\_idx} \in \{0, 1 \}$: 1 if large station built at position of $ds\_idx$; otherwise 0.

$x_{s\_idx,dis\_idx} \in \mathbb{R}^+$: Amount of the district's $dis\_idx$ demand met by rental station $s\_idx$. 

#### Objective function

- **Costs**. Minimize investment costs.
 

\begin{equation}
\text{Min} \quad CostTarget = 
\text{costS}\sum_{i = 0}^{N\_DYNAMIC} ysmall_i + \text{costM}\sum_{i = N\_DYNAMIC}^{N\_DYNAMIC * 2} ymed_i + \text{costL}\sum_{i = N\_DYNAMIC * 2}^{N\_DYNAMIC * 3} ylarge_i
\tag{0}
\end{equation}


- **Distance**. Minimize Distance from District to Stations supplying their demand.
\begin{equation}
\text{Min} \quad DistanceTarget = \sum_{s\_idx \in S\_IDX} \sum_{dis\_idx \in DIX\_IDX} x_{s\_idx, dis\_idx} * ||pos(s\_idx) - pos(dis\_idx)||
\tag{1}
\end{equation}


- **Combined**. Combine Targets.
\begin{equation}
\text{Min} \quad T = w1*CostTarget + w2*DistanceTarget
\end{equation}

#### Constraints

- **Demand**. The complete demand of a district $dis\_idx$ should be met.

\begin{equation}
\sum_{s\_idx  \in  S\_IDX} x_{s\_idx,dis\_idx} = demand_{dis\_idx} \quad \forall dis\_idx \in DIS\_IDX
\tag{1}
\end{equation}

- **Capacity**. Capacity of stations cannot be exceeded. 
\begin{equation}
\sum_{dis\_idx \in DIS\_IDX} x_{s\_idx,dis\_idx} \leq ysmall_{s\_idx}*capS + ymed_{s\_idx}*capM + ylarge_{s\_idx}*capL  \quad \forall s\_idx \in S\_IDX
\tag{2}
\end{equation}

- **Location**. At every possible location $ds\_idx$ a maximum of one charging station can be built. 

\begin{equation}
ysmall_{ds\_idx} + ymed_{ds\_idx} + ylarge_{ds\_idx} \leq 1 \quad \forall ds\_idx \in DS\_IDX
\tag{3}
\end{equation}

# Implementation

In [471]:
from itertools import product
from math import sqrt
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [472]:
DATE = "01/01/2019"
NUM_DYNAMIC_STATIONS = 30

In [473]:
# fixed stations
coords = pd.read_csv("koordinaten.csv").drop(columns="Station").rename(columns={"Terminal": "Station"})
df_fs = pd.read_csv("station_demand_per_day.csv", low_memory=False)
df_fs["Station"] = pd.to_numeric(df_fs["Station"], errors="coerce")
df_fs = pd.merge(df_fs, coords, on="Station")
df_fs = df_fs.where(df_fs["Day"] == DATE).dropna().drop(columns=["Day", "Docks", "Demand"])
df_fs.rename(columns={"Longitude": "Lon", "Latitude": "Lat"}, inplace=True)
df_fs["Station"] = range(len(df_fs))

In [474]:
# districts
zipcoords = pd.read_csv("zipcoords.csv", header=None, names=["District", "Lat", "Lon"])
df_dis = pd.merge(pd.read_csv("district_demand_per_day.csv"), zipcoords, on="District")
df_dis = df_dis.where(df_dis["Day"] == DATE).dropna().drop(columns=["Day"])

In [475]:
# dynamic stations
latlims = df_dis["Lat"].agg(["min", "max"])
df_ds = pd.DataFrame(columns=["Station", "Lat", "Lon"])
for i in range(NUM_DYNAMIC_STATIONS):
    df_ds.loc[i] = [i, np.random.uniform(*df_dis["Lat"].agg(["min", "max"])), np.random.uniform(*df_dis["Lon"].agg(["min", "max"]))]

In [478]:
df_dis = df_dis.head(25)
df_fs = df_fs.head(30)

In [486]:
# Costs
costS = 5
costM = 20
costL = 50

# Capacities
capS = 5
capM = 15
capL = 40

demandMod = 2

fs = {s:(x,y) for i, (s,x,y) in df_fs.iterrows()}
ds = {s:(x,y) for i, (s,x,y) in df_ds.iterrows()}

# district to coord
distr = {d:(x,y) for i, (d,x,y,dem) in df_dis.iterrows()}

# station to capacity (fixed for now, didn't find data @dataacquisition O.o)
capacity = {s:capM for i, (s,x,y) in df_fs.iterrows()}

# district to demand
demand = {d:dem*demandMod for i, (d,dem,x,y) in df_dis.iterrows()}

n_fixed = len(df_fs)
n_dynamic = len(df_ds)
n_total = n_fixed + n_dynamic


# Loss weighting
w1 = 1
w2 = 1

# Calculation of the distances between the districts and the locations of the charging stations
dists_fix = {}
for d_i, d_v in distr.items():
    for s_i, s_v in fs.items():
        dists_fix[(d_i, s_i)] = sqrt((d_v[0] - s_v[0])**2 + (d_v[1] - s_v[1])**2)

dists_dyn = {}
for d_i, d_v in distr.items():
    for s_i, s_v in ds.items():
        dists_dyn[(d_i, s_i)] = sqrt((d_v[0] - s_v[0])**2 + (d_v[1] - s_v[1])**2)

#####################################################
#                    MILP Model Formulation
#####################################################

# Model
m = gp.Model('eval_rental_positions')

# Decision variables
offsetidx = [range(0, n_dynamic), range(n_dynamic, n_dynamic*2), range(n_dynamic*2, n_dynamic*3)]
ys = m.addVars(offsetidx[0], vtype=GRB.BINARY, name='small')
ym = m.addVars(offsetidx[1], vtype=GRB.BINARY, name='medium')
yl = m.addVars(offsetidx[2], vtype=GRB.BINARY, name='large')

# Parameter
xdyn = m.addVars(dists_dyn.keys(), vtype=GRB.CONTINUOUS, name='AssignDyn', lb=0)
xfix = m.addVars(dists_fix.keys(), vtype=GRB.CONTINUOUS, name='AssignFix', lb=0)

# Objective function
m.setObjective(
    (costS * gp.quicksum(ys[i] for i in offsetidx[0]) + costM * gp.quicksum(ym[i] for i in offsetidx[1]) + costL * gp.quicksum(yl[i] for i in offsetidx[2])) 
    +
    gp.quicksum(w2 * xdyn[(i,j)]*dists_dyn[(i,j)] for i in distr.keys() for j in range(n_dynamic)) 
    +
    gp.quicksum(w2 * xfix[(i,j)]*dists_fix[(i,j)] for i in distr.keys() for j in range(n_fixed))
    ,
    GRB.MINIMIZE
)
# dyn and fixed stations supply all demand
m.addConstrs((gp.quicksum(xdyn[(i,j)] for j in range(n_dynamic)) + gp.quicksum(xfix[(i,j)] for j in range(n_fixed)) >= demand[i] for i in distr.keys()), name="demandConstr")
# dyn stations have enough capacity
m.addConstrs((gp.quicksum(xdyn[(d,i)] for d in distr.keys()) <= ys[i]*capS + ym[j]*capM + yl[k]*capL for (i,j,k) in zip(*offsetidx)), name="capacityFixConstr")
# fixed stations have enough capacity
m.addConstrs((gp.quicksum(xfix[(i,j)] for i in distr.keys()) <= capacity[j] for j in range(n_fixed)), name="capacityDynConstr")
# max one station per location
m.addConstrs((ys[i] + ym[j] + yl[k] <= 1 for (i,j,k) in zip(*offsetidx)), name="PhysicsConstr")

m.optimize()

post_ds = df_ds.copy()
post_ds["Type"] = pd.Series(df_ds.index).apply(lambda x: ys[x].X+2*ym[x+n_dynamic].X+3*yl[x+2*n_dynamic].X).map({
    0:"Ignored",
    1: "Built Small",
    2: "Built Medium",
    3: "Built Large"
})
post_fixed = df_fs.copy()
post_fixed["Type"] = "Prebuilt"
post_distr = df_dis.copy()
post_distr["Type"] = "District"
# create scatter trace for nodes
t1 = px.scatter(pd.concat([post_ds, post_fixed, post_distr]), x="Lat", y="Lon", color="Type",color_discrete_map={
            "Ignored": "grey",
            "Prebuilt": "orange",
            "Built Small": "lightgreen",
            "Built Medium": "green",
            "Built Large": "darkgreen",
            "District": "blue"}).data

fig = go.Figure(data=t1)
fig.show()
pass

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 115 rows, 1590 columns and 3180 nonzeros
Model fingerprint: 0xc06b5385
Variable types: 1500 continuous, 90 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [5e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 130389.60627
Presolve time: 0.01s
Presolved: 115 rows, 1590 columns, 3180 nonzeros
Variable types: 1500 continuous, 90 integer (90 binary)
Found heuristic solution: objective 130299.60627

Root relaxation: objective 1.301477e+05, 472 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   